# Description 

__Program__ : paccarPartsMLOps  
__Description__: predicting the PACCAR Parts regional hits  
__Team__: Cascade Consulting  
__Author__: Saran Pavuluri -- _with assistance from Kelly, Ben, Mohan, Jeremy, Joy and Dohoon_  


#### import required libraries:

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


#### Set the max columns and rows to max, to enable viewing of data

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### read the data file into a pandas dataframe

In [3]:
data = pd.read_csv("training_data_0101_0514.csv")

#### Checking if there is an inbalance in the dataset

In [4]:
data.value_counts('rhit_label')/len(data)

rhit_label
0    0.692619
1    0.307381
Name: count, dtype: float64

No imbalance exists in the data distribution, let us see how many columns have NA's and handle them

In [5]:
data.isna().sum()

ID                              0
suggestion_dt                   0
expid                           0
expid_desc                      0
item_id                         0
pdc                             0
vndr_concat                     0
desk                            0
velocity                        0
part_cost                      14
spq                             0
ord_min                         0
ord_mult                        0
ord_dollar_min                  0
lead_time                       0
on_hand                         0
on_order                        0
ss_units_left_pct             345
max_oh_left_pct                62
days_on_hand                    0
oh_oo_sug_dos                   0
oo_dos                          0
doh_less_ss                     0
doh_to_ltm                      0
oo_it_portion                   0
oh_5d_change                30506
min_on_hand_change_5d       19690
days_below_ss                   0
fcst_3m                         0
fcst_daily    

In [6]:
# checking which item_id are missing part_cost values

list = list(data[data['part_cost'].isna()]["item_id"])

data[data["item_id"].isin(list)][["item_id","part_cost"
                                 ]].sort_values(by=['item_id'])




,item_id,part_cost
45055,28243-6CN2,NaN
68816,I-BC-L9780-21,34.0
181487,I-BC-L9780-21,34.0
77218,I-BC-L9780-21,34.0
77129,I-BC-L9780-21,34.0
77017,I-BC-L9780-21,34.0
77016,I-BC-L9780-21,34.0
69047,I-BC-L9780-21,34.0
69000,I-BC-L9780-21,34.0
68954,I-BC-L9780-21,NaN


In [7]:
# Dictionary of known part_cost values
fill_values = {
    '28243-6CN2': 0.1,
    'Z14-8458-233352': 255.5,
    'I-BC-L9780-21': 34.0,
    'I-BC-L9780-32': 0.1,
    'I2789LL4402W': 40.0,
    'I33-9578': 3.1,
    'W3427X3148Z3N': 0.1,
    'XV4NM455W4': 24.6,
}

# Fill missing part_cost values
for item_id in list:
    if item_id in fill_values:
        data.loc[data['item_id'] == item_id, 'part_cost'] = fill_values[item_id]
    else:
        data.loc[data['item_id'] == item_id, 'part_cost'] = 0.1

In [8]:
round((data.isna().sum()*100)/len(data))

ID                           0.0
suggestion_dt                0.0
expid                        0.0
expid_desc                   0.0
item_id                      0.0
pdc                          0.0
vndr_concat                  0.0
desk                         0.0
velocity                     0.0
part_cost                    0.0
spq                          0.0
ord_min                      0.0
ord_mult                     0.0
ord_dollar_min               0.0
lead_time                    0.0
on_hand                      0.0
on_order                     0.0
ss_units_left_pct            0.0
max_oh_left_pct              0.0
days_on_hand                 0.0
oh_oo_sug_dos                0.0
oo_dos                       0.0
doh_less_ss                  0.0
doh_to_ltm                   0.0
oo_it_portion                0.0
oh_5d_change                12.0
min_on_hand_change_5d        8.0
days_below_ss                0.0
fcst_3m                      0.0
fcst_daily                   0.0
supplier_p

In [9]:
correlation_matrix = data.corr(numeric_only = True)

for row in correlation_matrix.index:
    for col in correlation_matrix.columns:
        correlation_value = correlation_matrix.loc[row, col]
        if (abs(correlation_value) > 0.7) & (abs(correlation_value) < 1):
            print(f"Row: {row}, Column: {col}, Correlation: {correlation_value}")
            

Row: ord_min, Column: ord_mult, Correlation: 0.9930256896635035
Row: ord_mult, Column: ord_min, Correlation: 0.9930256896635035
Row: on_hand, Column: fcst_3m, Correlation: 0.8739086373372763
Row: on_hand, Column: fcst_daily, Correlation: 0.8739086373547202
Row: on_hand, Column: dmd_rolling_90d, Correlation: 0.8011029205656477
Row: on_hand, Column: dmd_wkly_95pct, Correlation: 0.7175333937946996
Row: on_order, Column: fcst_3m, Correlation: 0.8924784458734565
Row: on_order, Column: fcst_daily, Correlation: 0.8924784458622969
Row: on_order, Column: dmd_rolling_90d, Correlation: 0.875674851123055
Row: on_order, Column: dmd_wkly_95pct, Correlation: 0.8820036093476467
Row: ss_units_left_pct, Column: max_oh_left_pct, Correlation: 0.8167377153463792
Row: max_oh_left_pct, Column: ss_units_left_pct, Correlation: 0.8167377153463792
Row: days_on_hand, Column: doh_to_ltm, Correlation: 0.8003150665981928
Row: doh_to_ltm, Column: days_on_hand, Correlation: 0.8003150665981928
Row: fcst_3m, Column: on_

#### Dropping identifiers and those that have high correlation factors

In [10]:
data.drop(['ID', 'suggestion_dt','item_id','vndr_concat', 
           'expid', 'expid_desc','ord_mult', 'fcst_daily',
           'on_hand', 'on_order','max_oh_left_pct','doh_to_ltm',
           'dmd_rolling_90d','dmd_wkly_95pct',
           'ltm_90pct_difference_wks','ltm_75pct'], 
          axis=1, inplace=True)

In [11]:
data.columns

Index(['pdc', 'desk', 'velocity', 'part_cost', 'spq', 'ord_min',
       'ord_dollar_min', 'lead_time', 'ss_units_left_pct', 'days_on_hand',
       'oh_oo_sug_dos', 'oo_dos', 'doh_less_ss', 'oo_it_portion',
       'oh_5d_change', 'min_on_hand_change_5d', 'days_below_ss', 'fcst_3m',
       'supplier_past_due_pct', 'ots_pct', 'early_ratio', 'on_time_ratio',
       'no_ship_ratio', 'dmd_fcst_portion', 'orders_12m', 'dmd_wkly_dos',
       'mdi_stockouts', 'network_avail', 'ltm_median', 'ltm_90pct',
       'rhit_within_2wks', 'rhit_label'],
      dtype='object')

#### Recheck the correlation factors

In [12]:
correlation_matrix = data.corr(numeric_only = True)

for row in correlation_matrix.index:
    for col in correlation_matrix.columns:
        correlation_value = correlation_matrix.loc[row, col]
        if (abs(correlation_value) > 0.75) & (abs(correlation_value) < 1):
            print(f"Row: {row}, Column: {col}, Correlation: {correlation_value}")
            

#### Checking data types to make the data ready for modelling

In [13]:
data.dtypes

pdc                       object
desk                      object
velocity                  object
part_cost                float64
spq                        int64
ord_min                    int64
ord_dollar_min             int64
lead_time                  int64
ss_units_left_pct        float64
days_on_hand             float64
oh_oo_sug_dos            float64
oo_dos                   float64
doh_less_ss                int64
oo_it_portion            float64
oh_5d_change             float64
min_on_hand_change_5d    float64
days_below_ss              int64
fcst_3m                  float64
supplier_past_due_pct    float64
ots_pct                  float64
early_ratio              float64
on_time_ratio            float64
no_ship_ratio            float64
dmd_fcst_portion         float64
orders_12m               float64
dmd_wkly_dos             float64
mdi_stockouts            float64
network_avail            float64
ltm_median               float64
ltm_90pct                float64
rhit_withi

In [14]:
object_columns = data.select_dtypes(include='object').columns
data[object_columns] = data[object_columns].astype('string')

In [15]:
data.dtypes

pdc                      string[python]
desk                     string[python]
velocity                 string[python]
part_cost                       float64
spq                               int64
ord_min                           int64
ord_dollar_min                    int64
lead_time                         int64
ss_units_left_pct               float64
days_on_hand                    float64
oh_oo_sug_dos                   float64
oo_dos                          float64
doh_less_ss                       int64
oo_it_portion                   float64
oh_5d_change                    float64
min_on_hand_change_5d           float64
days_below_ss                     int64
fcst_3m                         float64
supplier_past_due_pct           float64
ots_pct                         float64
early_ratio                     float64
on_time_ratio                   float64
no_ship_ratio                   float64
dmd_fcst_portion                float64
orders_12m                      float64


#### Converting velocity parameter into int, by using cat.codes

In [16]:
data.value_counts('velocity')

velocity
3    80326
2    51233
N    36771
9    24935
1    21208
M    14280
L    14000
T      685
A      656
S       25
8       15
E        3
Name: count, dtype: int64

In [17]:
data['velocity'] = np.where(data['velocity'] == '8', 4, 
                              np.where(data['velocity'] == '9', 5,
                                       np.where(data['velocity'] == 'A', 6, 
                                               np.where(data['velocity'] == 'E', 7,
                                                       np.where(data['velocity'] == 'L', 8,
                                                                np.where(data['velocity'] == 'M', 9,
                                                                         np.where(data['velocity'] == 'N', 10,
                                                                                  np.where(data['velocity'] == 'S', 11,
                                                                                           np.where(data['velocity'] == 'T', 12,
                                                                                                   data['velocity'])))))))))


In [18]:
data.value_counts('velocity')

velocity
3     80326
2     51233
10    36771
5     24935
1     21208
9     14280
8     14000
12      685
6       656
11       25
4        15
7         3
Name: count, dtype: int64

In [19]:
data['pdc'] = data['pdc'].astype('category').cat.codes
data['desk'] = data['desk'].astype('category').cat.codes


In [20]:
object_columns = data.select_dtypes(include='object').columns
data[object_columns] = data[object_columns].astype('int64')

In [21]:
data.dtypes

pdc                         int8
desk                        int8
velocity                   int64
part_cost                float64
spq                        int64
ord_min                    int64
ord_dollar_min             int64
lead_time                  int64
ss_units_left_pct        float64
days_on_hand             float64
oh_oo_sug_dos            float64
oo_dos                   float64
doh_less_ss                int64
oo_it_portion            float64
oh_5d_change             float64
min_on_hand_change_5d    float64
days_below_ss              int64
fcst_3m                  float64
supplier_past_due_pct    float64
ots_pct                  float64
early_ratio              float64
on_time_ratio            float64
no_ship_ratio            float64
dmd_fcst_portion         float64
orders_12m               float64
dmd_wkly_dos             float64
mdi_stockouts            float64
network_avail            float64
ltm_median               float64
ltm_90pct                float64
rhit_withi

In [22]:
round((data.isna().sum()*100)/len(data))

pdc                       0.0
desk                      0.0
velocity                  0.0
part_cost                 0.0
spq                       0.0
ord_min                   0.0
ord_dollar_min            0.0
lead_time                 0.0
ss_units_left_pct         0.0
days_on_hand              0.0
oh_oo_sug_dos             0.0
oo_dos                    0.0
doh_less_ss               0.0
oo_it_portion             0.0
oh_5d_change             12.0
min_on_hand_change_5d     8.0
days_below_ss             0.0
fcst_3m                   0.0
supplier_past_due_pct     2.0
ots_pct                   4.0
early_ratio               4.0
on_time_ratio             4.0
no_ship_ratio             4.0
dmd_fcst_portion          4.0
orders_12m                0.0
dmd_wkly_dos              2.0
mdi_stockouts            12.0
network_avail            12.0
ltm_median                5.0
ltm_90pct                 5.0
rhit_within_2wks          0.0
rhit_label                0.0
dtype: float64

In [23]:
print(data.describe())



                 pdc           desk       velocity      part_cost  \
count  244137.000000  244137.000000  244137.000000  244137.000000   
mean        3.983960       8.420883       4.546910     120.543420   
std         2.843953       4.938374       3.122622     323.798665   
min         0.000000       0.000000       1.000000       0.000000   
25%         1.000000       4.000000       2.000000       7.700000   
50%         3.000000       9.000000       3.000000      27.600000   
75%         6.000000      12.000000       8.000000      95.900000   
max         9.000000      16.000000      12.000000   16823.500000   

                 spq        ord_min  ord_dollar_min      lead_time  \
count  244137.000000  244137.000000   244137.000000  244137.000000   
mean        2.197123       4.631240        0.105302      43.569311   
std        10.454645      43.710326        3.422129      24.865670   
min         1.000000       0.000000        0.000000       7.000000   
25%         1.000000       0

#### Missing values are filled with 0.01, an insignificant valye, and then we shall check if summary statistics have changed much

In [24]:
data.fillna(0.01, inplace=True)

In [25]:
print(data.describe())

                 pdc           desk       velocity      part_cost  \
count  244137.000000  244137.000000  244137.000000  244137.000000   
mean        3.983960       8.420883       4.546910     120.543420   
std         2.843953       4.938374       3.122622     323.798665   
min         0.000000       0.000000       1.000000       0.000000   
25%         1.000000       4.000000       2.000000       7.700000   
50%         3.000000       9.000000       3.000000      27.600000   
75%         6.000000      12.000000       8.000000      95.900000   
max         9.000000      16.000000      12.000000   16823.500000   

                 spq        ord_min  ord_dollar_min      lead_time  \
count  244137.000000  244137.000000   244137.000000  244137.000000   
mean        2.197123       4.631240        0.105302      43.569311   
std        10.454645      43.710326        3.422129      24.865670   
min         1.000000       0.000000        0.000000       7.000000   
25%         1.000000       0

No significant change appeared in summary statistics

### Final columns for our analysis

In [26]:
data.columns

Index(['pdc', 'desk', 'velocity', 'part_cost', 'spq', 'ord_min',
       'ord_dollar_min', 'lead_time', 'ss_units_left_pct', 'days_on_hand',
       'oh_oo_sug_dos', 'oo_dos', 'doh_less_ss', 'oo_it_portion',
       'oh_5d_change', 'min_on_hand_change_5d', 'days_below_ss', 'fcst_3m',
       'supplier_past_due_pct', 'ots_pct', 'early_ratio', 'on_time_ratio',
       'no_ship_ratio', 'dmd_fcst_portion', 'orders_12m', 'dmd_wkly_dos',
       'mdi_stockouts', 'network_avail', 'ltm_median', 'ltm_90pct',
       'rhit_within_2wks', 'rhit_label'],
      dtype='object')

In [27]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix

X_DT= pd.DataFrame()

dataNew = data.drop(['rhit_label','rhit_within_2wks'], axis=1)

X_DT= dataNew.copy()

y_DT = data['rhit_label']

from sklearn.model_selection import train_test_split

# Splitting the dataset into training and testing sets
X_train_DT, X_test_DT, y_train_DT, y_test_DT = train_test_split(X_DT, y_DT, 
                                                                test_size= 0.3, 
                                                                stratify= y_DT,
                                                                random_state=42)

# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()

# X_train_DT = scaler.fit_transform(X_train_DT)

# X_test_DT = scaler.transform(X_test_DT)

"""
SINCE CLASS IMBALANCE DOES NOT EXIST, NO NEED TO DO oversampling
"""

# from imblearn.over_sampling import SMOTE

# # ... your code for data preparation (X_DT, y_DT) and train-test split (X_train_DT, X_test_DT, y_train_DT, y_test_DT)

# # Oversample the minority class (assuming class 1 is the minority)
# smote = SMOTE(sampling_strategy='minority')  # Oversample the minority class

# X_train_DT, y_train_DT = smote.fit_resample(X_train_DT, y_train_DT)


class_weight = {0: 1, 1: 1}

# Creating and fitting the decision tree model
dt_classifier = DecisionTreeClassifier(class_weight = class_weight, 
                                       criterion='gini',
                                       max_depth=10, max_features=None, max_leaf_nodes=None,
                                       min_samples_leaf=180, min_samples_split=2, min_weight_fraction_leaf=0.0, 
                                       random_state=100, splitter='best')

dt_classifier.fit(X_train_DT, y_train_DT)

# Predicting the test set results
y_pred_DT = dt_classifier.predict(X_test_DT)

# Evaluating the model
accuracy = accuracy_score(y_test_DT, y_pred_DT)
precision = precision_score(y_test_DT, y_pred_DT, average='binary') 
recall = recall_score(y_test_DT, y_pred_DT, average='binary') 

# Printing the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

confMatrix = metrics.confusion_matrix(y_test_DT, y_pred_DT)

print(f"confusion matrix: {confMatrix}")
# plt.figure(figsize=(20,20))
# plot_tree(dt_classifier, filled=True, feature_names= ['pdc', 'desk', 'velocity', 'part_cost', 'spq', 'ord_min',
#        'ord_dollar_min', 'lead_time', 'ss_units_left_pct', 'days_on_hand',
#        'oh_oo_sug_dos', 'oo_dos', 'doh_less_ss', 'oo_it_portion',
#        'oh_5d_change', 'min_on_hand_change_5d', 'days_below_ss', 'fcst_3m',
#        'supplier_past_due_pct', 'ots_pct', 'early_ratio', 'on_time_ratio',
#        'no_ship_ratio', 'dmd_fcst_portion', 'orders_12m', 'dmd_wkly_dos',
#        'mdi_stockouts', 'network_avail', 'ltm_median', 'ltm_90pct'], class_names=['no RHIT', 'RHIT'])

# plt.show()

Accuracy: 0.7917042134294531
Precision: 0.7163814180929096
Recall: 0.5336028072669124
confusion matrix: [[45973  4756]
 [10500 12013]]


In [28]:
!pip install xgboost

## Accuracy and Precision figures from this part will be our results

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Assuming 'data' is your DataFrame and has been previously defined
X_DT = pd.DataFrame()

# Dropping target variables from features
dataNew = data.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
X_DT = dataNew.copy()
y_DT = data['rhit_label']

# Splitting the dataset into training and testing sets
X_train_DT, X_test_DT, y_train_DT, y_test_DT = train_test_split(X_DT, y_DT, 
                                                                test_size=0.3, 
                                                                stratify=y_DT, 
                                                                random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(objective='binary:logistic', 
                               max_depth=10, 
                               min_child_weight=180, 
                               random_state=100, 
                               use_label_encoder=False, 
                               eval_metric='logloss')

# Train the XGBoost model
xgb_classifier.fit(X_train_DT, y_train_DT)

# Predicting the test set results
y_pred_DT = xgb_classifier.predict(X_test_DT)

# Evaluating the model
accuracy = accuracy_score(y_test_DT, y_pred_DT)
precision = precision_score(y_test_DT, y_pred_DT, average='binary')
recall = recall_score(y_test_DT, y_pred_DT, average='binary')

# Printing the evaluation metrics
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# Confusion matrix
confMatrix = confusion_matrix(y_test_DT, y_pred_DT)
print(f"Confusion Matrix:\n{confMatrix}")


Accuracy: 0.8248818983643265
Precision: 0.7674932346606285
Recall: 0.6172877892773064
Confusion Matrix:
[[46519  4210]
 [ 8616 13897]]


# Splitting the data to perform a hold-out dataset analysis on our own, to make sure model is robust enough

#### This is only an additional analysis to see how the model is performing

In [30]:
df_30, df_70 = train_test_split(data, test_size=0.7, random_state=82)

### Also checking if the model works similarly across PDC's and Desk's

In [31]:
# Assuming 'data' is your DataFrame and has been previously defined
X_DT = pd.DataFrame()

# Dropping target variables from features
dataNew = data.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
X_DT = dataNew.copy()
y_DT = data['rhit_label']

# Splitting the dataset into training and testing sets
X_train_DT, X_test_DT, y_train_DT, y_test_DT = train_test_split(X_DT, y_DT, 
                                                                test_size=0.3, 
                                                                stratify=y_DT, 
                                                                random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(objective='binary:logistic', 
                               max_depth=10, 
                               min_child_weight=180, 
                               random_state=100, 
                               use_label_encoder=False, 
                               eval_metric='logloss')

# Train the XGBoost model
xgb_classifier.fit(X_train_DT, y_train_DT)

# Predicting the test set results
y_pred_DT = xgb_classifier.predict(X_test_DT)

# Evaluating the model
accuracy = accuracy_score(y_test_DT, y_pred_DT)
precision = precision_score(y_test_DT, y_pred_DT, average='binary')
recall = recall_score(y_test_DT, y_pred_DT, average='binary')

# Printing the evaluation metrics
print(f"Overall Accuracy: {accuracy}")
print(f"Overall Precision: {precision}")
print(f"Overall Recall: {recall}")

# Confusion matrix for each unique value in 'pdc'
unique_pdc_values = X_test_DT['pdc'].unique()

confResults = []

for pdc_value in unique_pdc_values:
    # Selecting the subset of data where 'pdc' equals the current value
    subset_index = X_test_DT[X_test_DT['pdc'] == pdc_value].index
    y_test_subset = y_test_DT.loc[subset_index]
    
    # Predicting the subset of test data
    y_pred_subset = xgb_classifier.predict(X_test_DT.loc[subset_index])
    
    # Calculating confusion matrix
    conf_matrix = confusion_matrix(y_test_subset, y_pred_subset)
    accuracy = accuracy_score(y_test_subset, y_pred_subset)
    precision = precision_score(y_test_subset, y_pred_subset, average='binary')
    recall = recall_score(y_test_subset, y_pred_subset, average='binary')
    
    print(f"\nPDC Value: {pdc_value}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    
    confResults.append({
        'pdc_value': pdc_value,
        'conf_matrix': conf_matrix,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall
    })

confResults_df = pd.DataFrame(confResults)

# Optionally, display the DataFrame
print(confResults_df)


Overall Accuracy: 0.8248818983643265
Overall Precision: 0.7674932346606285
Overall Recall: 0.6172877892773064

PDC Value: 0
Confusion Matrix:
[[5436  688]
 [1185 2266]]
Accuracy: 0.8043864229765013
Precision: 0.7670954637779283
Recall: 0.6566212691973341

PDC Value: 9
Confusion Matrix:
[[3330  265]
 [ 626  865]]
Accuracy: 0.8248132127408573
Precision: 0.7654867256637168
Recall: 0.5801475519785378

PDC Value: 6
Confusion Matrix:
[[9744  616]
 [1592 2189]]
Accuracy: 0.8438582844211866
Precision: 0.7803921568627451
Recall: 0.5789473684210527

PDC Value: 7
Confusion Matrix:
[[5134  473]
 [1069 1547]]
Accuracy: 0.8124771981028822
Precision: 0.7658415841584159
Recall: 0.5913608562691132

PDC Value: 4
Confusion Matrix:
[[368  42]
 [ 55 186]]
Accuracy: 0.8509984639016898
Precision: 0.8157894736842105
Recall: 0.7717842323651453

PDC Value: 3
Confusion Matrix:
[[5954  647]
 [1259 2290]]
Accuracy: 0.8122167487684729
Precision: 0.7797071842015663
Recall: 0.6452521837137222

PDC Value: 2
Confusion 

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Splitting the data into 70% and 30% for training/initial testing and hold-out
df_30, df_70 = train_test_split(data, test_size=0.7, random_state=42)

# Prepare the training/initial testing dataset
X_70 = df_70.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_70 = df_70['rhit_label']

# Splitting df_70 into training and testing sets
X_train_70, X_test_70, y_train_70, y_test_70 = train_test_split(X_70, y_70, 
                                                                test_size=0.3, 
                                                                stratify=y_70, 
                                                                random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(objective='binary:logistic', 
                               max_depth=10, 
                               min_child_weight=180, 
                               random_state=100, 
                               use_label_encoder=False, 
                               eval_metric='logloss')

# Train the XGBoost model
xgb_classifier.fit(X_train_70, y_train_70)

# Predicting the test set results for df_70
y_pred_70 = xgb_classifier.predict(X_test_70)

# Evaluating the model on df_70
accuracy_70 = accuracy_score(y_test_70, y_pred_70)
precision_70 = precision_score(y_test_70, y_pred_70, average='binary')
recall_70 = recall_score(y_test_70, y_pred_70, average='binary')

# Printing the evaluation metrics for df_70
print("Evaluation on df_70 (70% data):")
print(f"Accuracy: {accuracy_70}")
print(f"Precision: {precision_70}")
print(f"Recall: {recall_70}")

# Confusion matrix for df_70
conf_matrix_70 = confusion_matrix(y_test_70, y_pred_70)
print(f"Confusion Matrix:\n{conf_matrix_70}")

# Evaluate the model on df_30 as hold-out data
X_30 = df_30.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_30 = df_30['rhit_label']

# Predicting the hold-out set results for df_30
y_pred_30 = xgb_classifier.predict(X_30)

# Evaluating the model on df_30
accuracy_30 = accuracy_score(y_30, y_pred_30)
precision_30 = precision_score(y_30, y_pred_30, average='binary')
recall_30 = recall_score(y_30, y_pred_30, average='binary')

# Printing the evaluation metrics for df_30
print("Evaluation on df_30 (30% hold-out data):")
print(f"Accuracy: {accuracy_30}")
print(f"Precision: {precision_30}")
print(f"Recall: {recall_30}")

# Confusion matrix for df_30
conf_matrix_30 = confusion_matrix(y_30, y_pred_30)
print(f"Confusion Matrix:\n{conf_matrix_30}")


Evaluation on df_70 (70% data):
Accuracy: 0.8162827439583374
Precision: 0.7592173350582148
Recall: 0.5931400416903544
Confusion Matrix:
[[32460  2978]
 [ 6441  9390]]
Evaluation on df_30 (30% hold-out data):
Accuracy: 0.8204284485465791
Precision: 0.7563525972565774
Recall: 0.6040768678160919
Confusion Matrix:
[[46635  4334]
 [ 8818 13454]]


In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Splitting the data into 70% and 30% for training/initial testing and hold-out
df_30, df_70 = train_test_split(data, test_size=0.7, random_state=42)

# Prepare the training/initial testing dataset
X_70 = df_70.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_70 = df_70['rhit_label']

# Splitting df_70 into training and testing sets
X_train_70, X_test_70, y_train_70, y_test_70 = train_test_split(X_70, y_70, 
                                                                test_size=0.3, 
                                                                stratify=y_70, 
                                                                random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(objective='binary:logistic', 
                               max_depth=10, 
                               min_child_weight=180, 
                               random_state=100, 
                               use_label_encoder=False, 
                               eval_metric='logloss')

# Train the XGBoost model
xgb_classifier.fit(X_train_70, y_train_70)

# Predicting the test set results for df_70
y_pred_70 = xgb_classifier.predict(X_test_70)

# Evaluating the model on df_70
accuracy_70 = accuracy_score(y_test_70, y_pred_70)
precision_70 = precision_score(y_test_70, y_pred_70, average='binary')
recall_70 = recall_score(y_test_70, y_pred_70, average='binary')

# Printing the evaluation metrics for df_70
print("Evaluation on df_70 (70% data):")
print(f"Accuracy: {accuracy_70}")
print(f"Precision: {precision_70}")
print(f"Recall: {recall_70}")

# Confusion matrix for df_70
conf_matrix_70 = confusion_matrix(y_test_70, y_pred_70)
print(f"Confusion Matrix:\n{conf_matrix_70}")

# Confusion matrix for each unique value in 'pdc' in the test set of df_70
unique_pdc_values = X_test_70['pdc'].unique()
confResults = []

for pdc_value in unique_pdc_values:
    # Selecting the subset of data where 'pdc' equals the current value
    subset_index = X_test_70[X_test_70['pdc'] == pdc_value].index
    y_test_subset = y_test_70.loc[subset_index]
    
    # Predicting the subset of test data
    y_pred_subset = xgb_classifier.predict(X_test_70.loc[subset_index])
    
    # Calculating confusion matrix
    conf_matrix = confusion_matrix(y_test_subset, y_pred_subset)
    accuracy = accuracy_score(y_test_subset, y_pred_subset)
    precision = precision_score(y_test_subset, y_pred_subset, average='binary')
    recall = recall_score(y_test_subset, y_pred_subset, average='binary')
    
    print(f"\nPDC Value: {pdc_value}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    
    confResults.append({
        'pdc_value': pdc_value,
        'conf_matrix': conf_matrix,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall
    })
    
confResults_df_70 = pd.DataFrame(confResults)

# Optionally, display the DataFrame
print("Confusion matrix results for each 'pdc' value on df_70:")
print(confResults_df_70)

# Evaluate the model on df_30 as hold-out data
X_30 = df_30.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_30 = df_30['rhit_label']

# Predicting the hold-out set results for df_30
y_pred_30 = xgb_classifier.predict(X_30)

# Evaluating the model on df_30
accuracy_30 = accuracy_score(y_30, y_pred_30)
precision_30 = precision_score(y_30, y_pred_30, average='binary')
recall_30 = recall_score(y_30, y_pred_30, average='binary')

# Printing the evaluation metrics for df_30
print("Evaluation on df_30 (30% hold-out data):")
print(f"Accuracy: {accuracy_30}")
print(f"Precision: {precision_30}")
print(f"Recall: {recall_30}")

# Confusion matrix for df_30
conf_matrix_30 = confusion_matrix(y_30, y_pred_30)
print(f"Confusion Matrix:\n{conf_matrix_30}")

# Confusion matrix for each unique value in 'pdc' in df_30
unique_pdc_values_30 = X_30['pdc'].unique()
confResults_30 = []

for pdc_value in unique_pdc_values_30:
    # Selecting the subset of data where 'pdc' equals the current value
    subset_index = X_30[X_30['pdc'] == pdc_value].index
    y_test_subset_30 = y_30.loc[subset_index]
    
    # Predicting the subset of hold-out data
    y_pred_subset_30 = xgb_classifier.predict(X_30.loc[subset_index])
    
    # Calculating confusion matrix
    conf_matrix_30 = confusion_matrix(y_test_subset_30, y_pred_subset_30)
    accuracy_30 = accuracy_score(y_test_subset_30, y_pred_subset_30)
    precision_30 = precision_score(y_test_subset_30, y_pred_subset_30, average='binary')
    recall_30 = recall_score(y_test_subset_30, y_pred_subset_30, average='binary')
    
    print(f"\nPDC Value: {pdc_value}")
    print(f"Confusion Matrix:\n{conf_matrix_30}")
    print(f"Accuracy: {accuracy_30}")
    print(f"Precision: {precision_30}")
    print(f"Recall: {recall_30}")
    
    confResults_30.append({
        'pdc_value': pdc_value,
        'conf_matrix': conf_matrix_30,
        'accuracy': accuracy_30,
        'precision': precision_30,
        'recall': recall_30
    })
    
confResults_df_30 = pd.DataFrame(confResults_30)

# Optionally, display the DataFrame
print("Confusion matrix results for each 'pdc' value on df_30:")
print(confResults_df_30)


Evaluation on df_70 (70% data):
Accuracy: 0.8162827439583374
Precision: 0.7592173350582148
Recall: 0.5931400416903544
Confusion Matrix:
[[32460  2978]
 [ 6441  9390]]

PDC Value: 1
Confusion Matrix:
[[4418  356]
 [ 811 1179]]
Accuracy: 0.8274689532820816
Precision: 0.7680781758957654
Recall: 0.592462311557789

PDC Value: 0
Confusion Matrix:
[[3754  480]
 [ 909 1575]]
Accuracy: 0.7932420363203334
Precision: 0.7664233576642335
Recall: 0.6340579710144928

PDC Value: 3
Confusion Matrix:
[[4232  472]
 [ 908 1516]]
Accuracy: 0.8063973063973064
Precision: 0.7625754527162978
Recall: 0.6254125412541254

PDC Value: 6
Confusion Matrix:
[[6857  483]
 [1153 1481]]
Accuracy: 0.8359735311810708
Precision: 0.7540733197556008
Recall: 0.5622627182991647

PDC Value: 5
Confusion Matrix:
[[1878  167]
 [ 387  418]]
Accuracy: 0.8056140350877193
Precision: 0.7145299145299145
Recall: 0.5192546583850932

PDC Value: 2
Confusion Matrix:
[[3809  364]
 [ 748 1183]]
Accuracy: 0.817824377457405
Precision: 0.764705882

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Splitting the data into 70% and 30% for training/initial testing and hold-out
df_30, df_70 = train_test_split(data, test_size=0.7, random_state=42)

# Prepare the training/initial testing dataset
X_70 = df_70.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_70 = df_70['rhit_label']

# Splitting df_70 into training and testing sets
X_train_70, X_test_70, y_train_70, y_test_70 = train_test_split(X_70, y_70, 
                                                                test_size=0.3, 
                                                                stratify=y_70, 
                                                                random_state=42)

# Initialize the XGBoost classifier
xgb_classifier = XGBClassifier(objective='binary:logistic', 
                               max_depth=10, 
                               min_child_weight=180, 
                               random_state=100, 
                               use_label_encoder=False, 
                               eval_metric='logloss')

# Train the XGBoost model
xgb_classifier.fit(X_train_70, y_train_70)

# Predicting the test set results for df_70
y_pred_70 = xgb_classifier.predict(X_test_70)

# Evaluating the model on df_70
accuracy_70 = accuracy_score(y_test_70, y_pred_70)
precision_70 = precision_score(y_test_70, y_pred_70, average='binary')
recall_70 = recall_score(y_test_70, y_pred_70, average='binary')

# Printing the evaluation metrics for df_70
print("Evaluation on df_70 (70% data):")
print(f"Accuracy: {accuracy_70}")
print(f"Precision: {precision_70}")
print(f"Recall: {recall_70}")

# Confusion matrix for df_70
conf_matrix_70 = confusion_matrix(y_test_70, y_pred_70)
print(f"Confusion Matrix:\n{conf_matrix_70}")

# Confusion matrix for each unique value in 'Desk' in the test set of df_70
unique_Desk_values = X_test_70['desk'].unique()
confResults = []

for Desk_value in unique_Desk_values:
    # Selecting the subset of data where 'Desk' equals the current value
    subset_index = X_test_70[X_test_70['desk'] == Desk_value].index
    y_test_subset = y_test_70.loc[subset_index]
    
    # Predicting the subset of test data
    y_pred_subset = xgb_classifier.predict(X_test_70.loc[subset_index])
    
    # Calculating confusion matrix
    conf_matrix = confusion_matrix(y_test_subset, y_pred_subset)
    accuracy = accuracy_score(y_test_subset, y_pred_subset)
    precision = precision_score(y_test_subset, y_pred_subset, average='binary')
    recall = recall_score(y_test_subset, y_pred_subset, average='binary')
    
    print(f"\nDesk Value: {Desk_value}")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    
    confResults.append({
        'Desk_value': Desk_value,
        'conf_matrix': conf_matrix,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall
    })
    
confResults_df_70 = pd.DataFrame(confResults)

# Optionally, display the DataFrame
print("Confusion matrix results for each 'Desk' value on df_70:")
print(confResults_df_70)

# Evaluate the model on df_30 as hold-out data
X_30 = df_30.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_30 = df_30['rhit_label']

# Predicting the hold-out set results for df_30
y_pred_30 = xgb_classifier.predict(X_30)

# Evaluating the model on df_30
accuracy_30 = accuracy_score(y_30, y_pred_30)
precision_30 = precision_score(y_30, y_pred_30, average='binary')
recall_30 = recall_score(y_30, y_pred_30, average='binary')

# Printing the evaluation metrics for df_30
print("Evaluation on df_30 (30% hold-out data):")
print(f"Accuracy: {accuracy_30}")
print(f"Precision: {precision_30}")
print(f"Recall: {recall_30}")

# Confusion matrix for df_30
conf_matrix_30 = confusion_matrix(y_30, y_pred_30)
print(f"Confusion Matrix:\n{conf_matrix_30}")

# Confusion matrix for each unique value in 'Desk' in df_30
unique_Desk_values_30 = X_30['desk'].unique()
confResults_30 = []

for Desk_value in unique_Desk_values_30:
    # Selecting the subset of data where 'Desk' equals the current value
    subset_index = X_30[X_30['desk'] == Desk_value].index
    y_test_subset_30 = y_30.loc[subset_index]
    
    # Predicting the subset of hold-out data
    y_pred_subset_30 = xgb_classifier.predict(X_30.loc[subset_index])
    
    # Calculating confusion matrix
    conf_matrix_30 = confusion_matrix(y_test_subset_30, y_pred_subset_30)
    accuracy_30 = accuracy_score(y_test_subset_30, y_pred_subset_30)
    precision_30 = precision_score(y_test_subset_30, y_pred_subset_30, average='binary')
    recall_30 = recall_score(y_test_subset_30, y_pred_subset_30, average='binary')
    
    print(f"\nDesk Value: {Desk_value}")
    print(f"Confusion Matrix:\n{conf_matrix_30}")
    print(f"Accuracy: {accuracy_30}")
    print(f"Precision: {precision_30}")
    print(f"Recall: {recall_30}")
    
    confResults_30.append({
        'Desk_value': Desk_value,
        'conf_matrix': conf_matrix_30,
        'accuracy': accuracy_30,
        'precision': precision_30,
        'recall': recall_30
    })
    
confResults_df_30 = pd.DataFrame(confResults_30)

# Optionally, display the DataFrame
print("Confusion matrix results for each 'Desk' value on df_30:")
print(confResults_df_30)


Evaluation on df_70 (70% data):
Accuracy: 0.8162827439583374
Precision: 0.7592173350582148
Recall: 0.5931400416903544
Confusion Matrix:
[[32460  2978]
 [ 6441  9390]]

Desk Value: 2
Confusion Matrix:
[[1721  116]
 [ 302  230]]
Accuracy: 0.8235542422963276
Precision: 0.6647398843930635
Recall: 0.4323308270676692

Desk Value: 1
Confusion Matrix:
[[2560  144]
 [ 441  366]]
Accuracy: 0.8333808031899743
Precision: 0.7176470588235294
Recall: 0.45353159851301117

Desk Value: 3
Confusion Matrix:
[[1574  116]
 [ 265  925]]
Accuracy: 0.8677083333333333
Precision: 0.8885686839577329
Recall: 0.7773109243697479

Desk Value: 9
Confusion Matrix:
[[3330  302]
 [ 596 1149]]
Accuracy: 0.8329923749302585
Precision: 0.791867677463818
Recall: 0.6584527220630373

Desk Value: 15
Confusion Matrix:
[[3472  552]
 [ 957 1608]]
Accuracy: 0.7709819395962969
Precision: 0.7444444444444445
Recall: 0.6269005847953216

Desk Value: 7
Confusion Matrix:
[[1051  121]
 [ 257  310]]
Accuracy: 0.7826336975273146
Precision: 0.

In [35]:
desk15 = data[data['desk'].isin([3,4])].copy()

In [36]:
# Evaluate the model on df_30 as hold-out data
X_30 = desk15.drop(['rhit_label', 'rhit_within_2wks'], axis=1)
y_30 = desk15['rhit_label']

# Predicting the hold-out set results for df_30
y_pred_30 = xgb_classifier.predict(X_30)

# Evaluating the model on df_30
accuracy_30 = accuracy_score(y_30, y_pred_30)
precision_30 = precision_score(y_30, y_pred_30, average='binary')
recall_30 = recall_score(y_30, y_pred_30, average='binary')

# Printing the evaluation metrics for df_30
print("Evaluation on df_30 (30% hold-out data):")
print(f"Accuracy: {accuracy_30}")
print(f"Precision: {precision_30}")
print(f"Recall: {recall_30}")

# Confusion matrix for df_30
conf_matrix_30 = confusion_matrix(y_30, y_pred_30)
print(f"Confusion Matrix:\n{conf_matrix_30}")

Evaluation on df_30 (30% hold-out data):
Accuracy: 0.8546027272350478
Precision: 0.8581025500411297
Recall: 0.7167888227210261
Confusion Matrix:
[[14360  1035]
 [ 2473  6259]]


# Note: 

Other approached tried include:
1. Using KNN classifier to fill in the missing values (found to be compute intensive, so had to drop it)
2. Decision Tree analysis - to understand the relationships, and if all PDC's and Desk's, which we view as key entities in this analysis, have similar results.
3. Since we observed that some Desk's and PDC's performed poorly when we used a simple decision tree analysis, we tried the following:
    * And there was low recall for some categories, so we tried assigning class weights to penalise this behaviour, but that did not result in desired results in train case itself. So, had to leave this approach
4. Next approach was to use clustering to identify different types of subsets and apply different models to them to improve accuracy and precision numbers. We have applied selectively Decision Tree and Random Forest algorithms to try and maximise the metrics, but that did not yield any better results.
5. Finally, we tried Xgboost algorithm to predict values, which resulted in better results. But, we wanted to be sure of the results, so we performed a hold-out analysis separately to see if the results fluctuated or remained the same. We tried different usecases such as single desk, multiple desks, single PDC, multiple PDC's and also tried various skewed subsets and found metrics to be robust enough.